In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import np_utils
import os
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.python.keras.applications.resnet import ResNet50
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from Training import GetDatasets
import tensorflow as tf
import tensorflow_datasets as tfds
from Nets_original import GetNeuralNetworkModel

# Preparing The Data

# Path de dataset

In [10]:
trainBatchSize = testBatchSize = 32
trainSet,validSet,testSet,x_train,x_valid,x_test = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize,return_size=True)

In [14]:
x_train

4000

In [15]:
x_valid

250

In [16]:
x_test

750

In [3]:
width_shape = 224
height_shape = 224
num_classes = 8
epochs = 50


# Transfer Learning modelo Resnet50 - fine tune

In [7]:
from keras.applications.resnet50 import ResNet50

image_input = Input(shape=(width_shape, height_shape, 3))

m_Resnet50 = ResNet50(input_tensor=image_input, include_top=False,weights='imagenet')

m_Resnet50.summary()

last_layer = m_Resnet50.layers[-1].output  #avg_pool

x = Flatten(name='flatten')(last_layer)
x = Dense(496, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)       # operaciones aleatoria para eliminar el 30% de estas
x = Dense(496, activation='relu', name='fc2')(x)
x = Dropout(0.3)(x)
out = Dense(8, activation='softmax', name='output')(x)
custom_model = Model(image_input, out)
custom_model.summary()
#
## freeze all the layers except the dense layers
for layer in custom_model.layers[:-6]:
    layer.trainable = False
custom_model.summary()
    


custom_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
#custom_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Training The Model

In [8]:
# Early Stopping
# --------------
#x_train=4000
#x_valid=250
#x_test=750

earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    
custom_model.fit(x=trainSet,epochs=1,
            steps_per_epoch  =int(np.ceil(x_train / trainBatchSize)),
            validation_data  =validSet,
            validation_steps =int(np.ceil(x_valid/ trainBatchSize)), 
            callbacks=[earlyStop])

 31/125 [======>.......................] - ETA: 3:57 - loss: 2.3018 - accuracy: 0.1331

KeyboardInterrupt: 

In [ ]:
custom_model.save("ResNet50.h5")

In [ ]:
(OrigLoss,OrigAcc) = ResNet50.evaluate(test_set)

# Dibujar la red

In [ ]:
import keras 

keras.utils.plot_model(model,show_shape=True)

# Evaluating The Model

In [ ]:


fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()
     


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()